## Kamodo: Tracking Changes

### Changes Overview
- add init file to flythrough
- add the path of kamodo codes to `SingleSatelliteFlythrough.py`
- add empty `__init__.py` file
- install `spacepy` package 
   ```
   pip install --upgrade numpy
   pip install --upgrade spacepy
   ```
- Install `astropy`  
    ```
    pip install astropy
    ```
- Corrected the reference dates in `TIEGCM_4D.py`.  The units on the "time" variable gave the wrong year so I grabbed the year from a different variable and used the given reference day (which seemed to be correct). I then input this date (YEAR+DOY) as the reference date for the times. 

- removed the missing value warnings from netcdf reading of TIEGCM to limit unnecessary printing to the console in GEODYN
    ```
    import warnings
    warnings.filterwarnings("ignore")
    ```
    
    
    
### Fixing Errors in TIEGCM Kamodo Reader
1. Error with timing in these files. 
    - `tiegcm_4D.py`
        - `l.172` added correct reference date
        - `l.178`added reference date to be passed into the timing functions


2. Error with coordinate systems
    - `tiegcm_4D.py`       
        - `l.33` *Note*: I don't think these should be put into geodetic coordinates but I guess I dont know how this is intended to fit into the other work yall are doing.  Altitude should not be returned in RE I don't think.
        - `l.73`  *Note*: I only made the following changes for the GEODYN/TIEGCM case. I left the requested coordinate system in the Variables as GDZ but I don't think this is correct.  The SPH coordinate system option was doing confusing things.
   - `SF_utilities.py`
      -  `l.916-920`, `Single_FlyAway_GEODYN`  The coordinate_systems() function was found to have the error because the altitudes after this were returned to be WAY too high.   
    
      - `l.494`, `coordinate_systems()`,  Somewhere in the ConvertCoord() func, the Altitude is returned multiplied by the earth's radius. (Alt*Re).  I wasn't able to find where this happens.  I'm not sure why this happened, but I'm returning the coordinates to the original given units for the TIEGCM case.
      - `l.505-506`   Return the C2 and C3 coordinates to the requested units. This effectively circumvents the Coordinate transformation.
 

## Pygeodyn and FORTRAN: Tracking changes

1. **Add new models to be accepted as inputs to pygeodyn.**
 1. in `/data/geodyn_proj/pygeodyn/pygeodyn_develop/util_dir/util_classtools.py`, add the additional models to the `geodyn_modify_inputs()` func.  This writes the file that is read into geodyn at each runtime with the model options.
    ```python
       def geodyn_modify_inputs(self, options_in, density_model):
           if options_in['DRHODZ_update']== True:
                drhodz_val = '1'
           elif options_in['DRHODZ_update']== False:
                drhodz_val = '0'
           else:
                sys.exit("DRHODZ option is in incorrect format")
           if density_model== 'msis86':
                model_val = '0'
           elif density_model== 'msis00':
                model_val = '1'
           elif density_model== 'msis2':
                model_val = '2'
           elif density_model== 'ctipe':
                model_val = '3'
           elif density_model== 'tiegcm':
                model_val = '4'
           elif density_model== 'gitm':
                model_val = '5'
           elif density_model== 'jaachia71':
                model_val = '0'        
           elif density_model== 'dtm87':
                model_val = '0'
           else:
                sys.exit("Density Model Option (DEN_DIR) is in incorrect format")

           file1 = open("/data/geodyn_proj/pygeodyn/pygeodyn_develop/geodyn_options.txt","w+")
           file1.writelines(drhodz_val+'\n') # first value is for DrhoDz
           file1.writelines(model_val +'\n') # 2nd values is for model switching
           file1.writelines('0'+'\n')
           file1.writelines('0'+'\n')
           file1.writelines('0'+'\n')
           file1.writelines('0'+'\n')
           file1.close()
        ```
        
  2. add model cases to ``set_density_model_setup_params()``
 
***
  
2. **Add new models to `DRAG.f90`**
  - ``/data/geodyn_proj/geodyn_code/IIE/Kamodo_pygeodyn_MODS/DRAG.f90``
  - add a case such that values greater than 3 call KamodoModels.f90.  add conditionals that identify the model.
      ```fortran
              case(3:)   ! Use Kamodo if case is greaterthan or equal to one
                 
                 if(choose_model.eq.3)then
                     kamodo_model='CTIPE'
                 if(choose_model.eq.4)then
                     kamodo_model='TIEGCM'
                 if(choose_model.eq.5)then
                     kamodo_model='GITM'
                 endif
                 
                 !!!!! Inputs for the Kamodo Reader
                 !          File_dir is the directory where the data is located
                 !          rho is the variable name
                 !          ilev means the variable depends on the pressure level for the CTIPe and TIEGCM model.
                 !          sat_time is the satellite timestamp in UTC since Jan 1 1970
                 !          sat_height: is satellite altitude above the ground (in km)
                 !          sat_lat satellite latitude
                 !          sat_lon satellite longitude               
                
                 !   sat_time   -- !   MJDSEC   I    S    TIME IN INTEGRAL SECONDS FROM GEODYN REF. TIME (
                                   ! will need to correct difference in ref times
                 !   sat_height -- ALTI
                 !   sat_lat    -- XLATD
                 !   sat_lon    -- XLOND
!                                                                       &
                if(kentry.eq.1)then
                  WRITE(6,*) 'CHECK-- DRAG.f90: Running w/ KamodoModels'
                endif
!                                                                       &
                 IJDSEC=MJDSEC+FSEC
                 CALL MJDYMD(IJDSEC,IYMD,IHMS,4)
                 IYR=(IYMD/10000.D0)+0.5D0
                CALL KamodoModels(MJDSEC,FSEC,ALTI,XLATD,XLOND,         &
             &         PHI,XLAMB,RHO,DRHODZ, DAY,IYR)
                C(1)=DRHODZ          
                if(kentry.eq.1)then
                  WRITE(6,*) 'CHECK-- DRAG.f90: Running w/ KamodoModels'
                endif
          end select
      ```
      
      
3. **Made many changes to ``KamodoModels.f90``. It is commented well enough to use as a template.**
